# Alpaca Lora Training

详细思路见：[https://github.com/unit-mesh/unit-minions](https://github.com/unit-mesh/unit-minions)

## Setup LFS

In [ ]:
!apt update
!apt install git-lfs
# check install
!git lfs install

## Clone to Local

In [10]:
!git clone https://huggingface.co/decapoda-research/llama-7b-hf

Cloning into 'llama-7b-hf'...
remote: Enumerating objects: 91, done.
remote: Counting objects: 100% (91/91), done.
remote: Compressing objects: 100% (89/89), done.
remote: Total 91 (delta 6), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (91/91), done.
Filtering content: 100% (34/34), 12.55 GiB | 42.81 MiB/s, done.


# Traing

In [ ]:
!git clone https://github.com/tloen/alpaca-lora

In [2]:
!cd alpaca-lora && !pip install -r requirements.txt

[Errno 2] No such file or directory: 'alpaca-lora'
/output/alpaca-lora
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
  Cloning https://github.com/huggingface/transformers.git (to revision c612628045822f909020f7eb6784c79700813eda) to /tmp/pip-req-build-c1w6a_2v
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-c1w6a_2v
  Running command git rev-parse -q --verify 'sha^c612628045822f909020f7eb6784c79700813eda'
  Running command git fetch -q https://github.com/huggingface/transformers.git c612628045822f909020f7eb6784c79700813eda
  Running command git checkout -q c612628045822f909020f7eb6784c79700813eda
  Resolved https://github.com/huggingface/transformers.git to commit c612628045822f909020f7eb6784c79700813eda
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/huggingface/peft.git (to revision

## Training in Local

In [ ]:
### Normal Traing

In [11]:
!cd alpaca-lora && python finetune.py \
    --base_model '../llama-7b-hf' \
    --data_path '../sql-train-8k.jsonl' \
    --output_dir './lora-alpaca'


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
/usr/local/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/usr/local/nvidia/lib64'), PosixPath('/usr/local/nvidia/lib')}
  warn(msg)
/usr/local/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: /usr/local/nvidia/lib:/usr/local/nvidia/lib64 did not contain libcudart.so as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('home/81fa1036-fb46-4b3c-9dd5-96e8dbc216d2')}
  warn(msg)
/usr/local/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.

# 评估

In [1]:
import torch
from peft import PeftModel
import transformers
import gradio as gr

assert (
    "LlamaTokenizer" in transformers._import_structure["models.llama"]
), "LLaMA is now in HuggingFace's main branch.\nPlease reinstall it: pip uninstall transformers && pip install git+https://github.com/huggingface/transformers.git"
from transformers import LlamaTokenizer, LlamaForCausalLM, GenerationConfig

tokenizer = LlamaTokenizer.from_pretrained("./llama-7b-hf")

BASE_MODEL = "./llama-7b-hf"
LORA_WEIGHTS = "./alpaca-lora/lora-alpaca"

if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

try:
    if torch.backends.mps.is_available():
        device = "mps"
except:
    pass

if device == "cuda":
    model = LlamaForCausalLM.from_pretrained(
        BASE_MODEL,
        load_in_8bit=False,
        torch_dtype=torch.float16,
        device_map="auto",
    )
    model = PeftModel.from_pretrained(
        model, LORA_WEIGHTS, torch_dtype=torch.float16, force_download=True
    )
elif device == "mps":
    model = LlamaForCausalLM.from_pretrained(
        BASE_MODEL,
        device_map={"": device},
        torch_dtype=torch.float16,
    )
    model = PeftModel.from_pretrained(
        model,
        LORA_WEIGHTS,
        device_map={"": device},
        torch_dtype=torch.float16,
    )
else:
    model = LlamaForCausalLM.from_pretrained(
        BASE_MODEL, device_map={"": device}, low_cpu_mem_usage=True
    )
    model = PeftModel.from_pretrained(
        model,
        LORA_WEIGHTS,
        device_map={"": device},
    )


def generate_prompt(instruction, input=None):
    if input:
        return f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction:
{instruction}
### Input:
{input}
### Response:"""
    else:
        return f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.
### Instruction:
{instruction}
### Response:"""

if device != "cpu":
    model.half()
model.eval()
if torch.__version__ >= "2":
    model = torch.compile(model)


def evaluate(
    instruction,
    input=None,
    temperature=0.1,
    top_p=0.75,
    top_k=40,
    num_beams=4,
    max_new_tokens=128,
    **kwargs,
):
    prompt = generate_prompt(instruction, input)
    inputs = tokenizer(prompt, return_tensors="pt")
    input_ids = inputs["input_ids"].to(device)
    generation_config = GenerationConfig(
        temperature=temperature,
        top_p=top_p,
        top_k=top_k,
        num_beams=num_beams,
        **kwargs,
    )
    with torch.no_grad():
        generation_output = model.generate(
            input_ids=input_ids,
            generation_config=generation_config,
            return_dict_in_generate=True,
            output_scores=True,
            max_new_tokens=max_new_tokens,
        )
    s = generation_output.sequences[0]
    output = tokenizer.decode(s)
    print(output.split("### Response:")[1].strip())


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching /usr/local/cuda/lib64...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 117
CUDA SETUP: Loading binary /usr/local/lib/python3.8/site-packages/bitsandbytes/libbitsandbytes_cuda117.so...


/usr/local/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/usr/local/nvidia/lib'), PosixPath('/usr/local/nvidia/lib64')}
  warn(msg)
/usr/local/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: /usr/local/nvidia/lib:/usr/local/nvidia/lib64 did not contain libcudart.so as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//10.111.0.1'), PosixPath('443'), PosixPath('tcp')}
  warn(msg)
/usr/local/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('http'), PosixPath('//alchemist-experience'), PosixPath('7890')}
  warn(msg)
/usr/loc

Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]

In [10]:
## 4K

evaluate("text to sql", "谁是最美丽的人", 0.1, 0.75, 40, 4, 512)
evaluate("text to sql", "小明今年几岁", 0.1, 0.75, 40, 4, 512)
evaluate("text to sql", "What hand guard system is used with a gas piston commando?", 0.1, 0.75, 40, 4, 512)

SELECT MOST BEAUTIFUL PERSON
SELECT 年龄
SELECT Hand Guard System FROM table WHERE Gas Piston Commando


In [2]:
# 8k

evaluate("text to sql", "谁是最美丽的人", 0.1, 0.75, 40, 4, 512)
evaluate("text to sql", "小明今年几岁", 0.1, 0.75, 40, 4, 512)
evaluate("text to sql", "What hand guard system is used with a gas piston commando?", 0.1, 0.75, 40, 4, 512)

SELECT MOST BEAUTIFUL FROM table WHERE BEAUTIFUL = 最美丽的人
SELECT MIN Year FROM table
SELECT Hand guard system FROM table WHERE Gas piston = Commando


In [ ]:
# 56k

evaluate("text to sql", "谁是最美丽的人", 0.1, 0.75, 40, 4, 512)
evaluate("text to sql", "小明今年几岁", 0.1, 0.75, 40, 4, 512)
evaluate("text to sql", "What hand guard system is used with a gas piston commando?", 0.1, 0.75, 40, 4, 512)

# SELECT Name FROM table WHERE 最美丽的人 = 谁是最美丽的人
# SELECT Year FROM table WHERE Name = 小明今年几岁
# SELECT Hand guard system FROM table WHERE Gas piston commando = yes